In [3]:
using FermiCG
using Printf
using Test
using JLD2 

@load "../test/_testdata_cmf_h12_64bit.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);

v_bst = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases)
v_tps = FermiCG.TPSCIstate(clusters, FockConfig(init_fspace))
FermiCG.eye!(v_tps)

s_bst = FermiCG.build_compressed_1st_order_state(v_bst, cluster_ops, clustered_ham, nbody=4, thresh=1e-24)
s_tps = FermiCG.open_matvec_thread(v_tps, cluster_ops, clustered_ham, thresh=1e-32)

@show size(s_bst);
@show size(s_tps);





 Remove duplicates
 Number of terms reduced from  2525 to  1105
IDX001:DIM0016:001|002|
IDX002:DIM0016:003|004|
IDX003:DIM0256:005|006|007|008|
IDX004:DIM0016:009|010|
IDX005:DIM0016:011|012|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.161381 seconds   6.24e-02 Gb GC: 3.8e-02 sec
 Add results together:                               0.024873 seconds   1.74e-02 Gb GC: 0.0e+00 sec
 In open_matvec_thread
 Setup threaded jobs:                                0.000209 seconds (1.58 k allocations: 145.469 KiB)
 Number of jobs:    525
 Number of threads: 1
 Compute matrix-vector:                              0.016953 seconds (78.87 k allocations: 6.587 MiB)
 Now collect thread results :                        0.000206 seconds (1.06 k allocations: 64.359 KiB)
size(s_bst) = (16278, 1)
size(s_tps) = (49378, 1)


# BST PT2

In [4]:
e_var = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);
e_pt = FermiCG.compute_pt2_energy(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-6, max_iter=100, tol=1e-12, verbose=0);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  2.446945 seconds (3.06 M allocations: 160.227 MiB, 3.98% gc time, 99.91% compilation time)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.985783 seconds
 Compute <S^2>:                                      0.003907 seconds (1.99 k allocations: 202.131 KiB, 88.70% compilation time)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 |== Compute PT2 Energy ============================================
 H0          : Hcmf
 max_iter    : 100
 nbody       : 4
 thresh_foi  : 1.0e-6
 max_number  : nothing
 tol         : 1.0e-12
 opt_ref     : true
 verbose     : 0

 Length of Reference:                                       1
 Solve zeroth-order problem:

In [5]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-33)
    
# FermiCG.project_out!(s_bst, v, thresh=1e-32)
# @show size(s_bst)
# s_bst = FermiCG.compress(s_bst, thresh=1e-32)
# @show size(s_bst)
v_pt, e_pt = FermiCG.hylleraas_compressed_mp2(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.023180 seconds (3.98 k allocations: 7.854 MiB, 81.80% gc time)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.023561 seconds
 Compute <S^2>:                                      0.000733 seconds (1.29 k allocations: 158.344 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.126798 seconds   6.24e-02 Gb GC: 0.0e+00 sec
 Add results together:                               0.042668 seconds   1.76e-02 Gb GC: 1.7e-02 sec
 Length of input      FOIS:                             16

In [6]:
e_pt, v_pt = FermiCG.do_fois_pt2(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-3, max_iter=50, tol=1e-9);

 |== Solve for BST PT1 Wavefunction ================================
 H0          : Hcmf
 max_iter    : 50
 nbody       : 4
 thresh_foi  : 0.001
 tol         : 1.0e-9
 opt_ref     : true
 verbose     : true

 Length of Reference:                                       1
 Solve zeroth-order problem:                       
 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.001821 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001210 seconds
 Compute <S^2>:                                      0.000684 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Diagonalization time:                               0.006731 seco

# Pseudo-Canonicalization

In [23]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-33)
    
FermiCG.pseudo_canon_pt1(s_bst, v, cluster_ops, clustered_ham, verbose=1);
# v_pt, e_pt = FermiCG.pseudo_canon_pt1(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);


 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.001866 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.000993 seconds
 Compute <S^2>:                                      0.000330 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.137974 seconds   6.24e-02 Gb GC: 1.6e-02 sec
 Add results together:                               0.026049 seconds   1.76e-02 Gb GC: 0.0e+00 sec
 Compute <X|H|0>:                                    0.026720 seconds   1.

In [20]:
Revise.retry()